### dynamoDB
This notebook adds data to the dynamoDB database

In [2]:
import boto3
import os
import json
import csv
from dotenv import load_dotenv
import numpy as np
import pandas as pd

In [3]:
# Load the environment variables from the .env file
load_dotenv()

True

In [4]:
# Creating the DynamoDB Client
# make sure to set the AWS_ACCESS_KEY_ID and
# AWS_SECRET_ACCESS_KEY environment variables
# OR pass credentials when creating the client:
dynamodb_client = boto3.client('dynamodb', region_name="us-east-1")

TABLE_NAME = "fumehoods"

dyn_resource = boto3.resource("dynamodb", region_name="us-east-1")
TABLE_NAME = "fumehoods"
table = dyn_resource.Table(TABLE_NAME)

In [5]:
def format_item(row):
    formatted_row = {}
    for key, value in row.items():
        if type(value) == str:
            formatted_row[key] = {'S': str(value)}
        if type(value) == int:
            formatted_row[key] = {'N': value}
    return {'M':formatted_row}

def read_fumehood_csv(path="./Point Names/biotech point names.csv"):
    names_df = pd.read_csv(path)
    names_df['name'] = names_df['building'].str.capitalize() + \
                            ".Floor_" + names_df['floor'].astype(str) + \
                            ".Lab_" + names_df['lab'] + \
                            ".Hood_" + names_df['hood'].astype(str)
    return names_df

def get_buildings_dict(buildings_columns = ["external_temp_sensor"]):
    buildings_names_df = read_fumehood_csv().copy().groupby('building').first().reset_index()
    buildings_names_df['building_name'] = buildings_names_df['building'].str.capitalize()
    buildings_names_df = buildings_names_df.set_index('building_name')[buildings_columns]

    buildings = {
                "id": "buildings",
                "map": buildings_names_df.to_dict('index')
    }

    return buildings

def get_labs_dict(labs_columns = ["building", "floor", "lab", "room_occ_sensor", "internal_temp_sensor"]):
    labs_names_df = read_fumehood_csv().copy().groupby(['building','floor','lab']).first().reset_index()
    labs_names_df['lab_name'] = labs_names_df['building'].str.capitalize() + \
                                        ".Floor_" + labs_names_df['floor'].astype(str) + \
                                        ".Lab_" + labs_names_df['lab'].str.lower()
    labs_names_df = labs_names_df.set_index('lab_name')[labs_columns]

    # TODO: Hardcoded
    labs_names_df.rename(index={"Biotech.Floor_G.Lab_g54":"Biotech.GroundFloor.Lab_g54"}, inplace=True)

    count = read_fumehood_csv().groupby(['building','floor','lab'])['hood'].count()
    count.name = 'hood_count'

    labs_names_df = labs_names_df.merge(count, left_on=['building','floor','lab'], right_index=True)

    labs = {
        "id": "labs",
        "map": labs_names_df.to_dict('index')
    }

    return labs

def get_hoods_dict(hoods_columns = ["building", "floor", "lab", "hood", "flow_sensor","sash_position_sensor"]):
    hoods_names_df = read_fumehood_csv().set_index('name')[hoods_columns]

    #hoods_data = hoods_names_df.apply(format_item, axis=1).to_dict()

    hoods = {
                "id": "hoods",
                "map": hoods_names_df.to_dict('index')
    }

    return hoods

In [6]:
get_hoods_dict()

{'id': 'hoods',
 'map': {'Biotech.Floor_G.Lab_G54.Hood_1': {'building': 'biotech',
   'floor': 'G',
   'lab': 'G54',
   'hood': 1,
   'flow_sensor': '#biotech/biotech_ground_floor/ground_floor_fume_hood_lab_spaces/lab_g54_control/hoodvalve_flow',
   'sash_position_sensor': '#biotech/biotech_ground_floor/ground_floor_fume_hood_lab_spaces/lab_g54_control/hood_sash'},
  'Biotech.Floor_1.Lab_141.Hood_1': {'building': 'biotech',
   'floor': '1',
   'lab': '141',
   'hood': 1,
   'flow_sensor': '#biotech/biotech_1st_floor/first_floor_fume_hood_lab_spaces/lab_141_control/hoodvalve_flow',
   'sash_position_sensor': '#biotech/biotech_1st_floor/first_floor_fume_hood_lab_spaces/lab_141_control/hood_sash'},
  'Biotech.Floor_1.Lab_143.Hood_1': {'building': 'biotech',
   'floor': '1',
   'lab': '143',
   'hood': 1,
   'flow_sensor': '#biotech/biotech_1st_floor/first_floor_fume_hood_lab_spaces/lab_143_control/hoodvalve_flow',
   'sash_position_sensor': '#biotech/biotech_1st_floor/first_floor_fume_hoo

In [7]:
get_buildings_dict()

{'id': 'buildings',
 'map': {'Biotech': {'external_temp_sensor': '#biotech/ground_flr_mech/building_hydronic_heating_syatems/reheat_heat_exchanger/oat'}}}

In [8]:
with table.batch_writer() as writer:
    writer.put_item(Item=get_buildings_dict())

response = dynamodb_client.get_item(
    TableName=TABLE_NAME, Key={"id": {"S": "buildings"}}
)
print("buildings:")
display(response["Item"])

buildings:


{'id': {'S': 'buildings'},
 'map': {'M': {'Biotech': {'M': {'external_temp_sensor': {'S': '#biotech/ground_flr_mech/building_hydronic_heating_syatems/reheat_heat_exchanger/oat'}}}}}}

In [9]:
with table.batch_writer() as writer:
    writer.put_item(Item=get_labs_dict())

response = dynamodb_client.get_item(
    TableName=TABLE_NAME, Key={"id": {"S": "labs"}}
)
print("Labs:")
display(response["Item"])

Labs:


{'id': {'S': 'labs'},
 'map': {'M': {'Biotech.Floor_2.Lab_203b': {'M': {'room_occ_sensor': {'S': '#biotech_lab_203b_hvac/occ_sensor'},
     'internal_temp_sensor': {'S': '#biotech_lab_203b_hvac/zone_temp'},
     'hood_count': {'N': '1'},
     'floor': {'S': '2'},
     'lab': {'S': '203B'},
     'building': {'S': 'biotech'}}},
   'Biotech.Floor_2.Lab_227b': {'M': {'room_occ_sensor': {'S': '#biotech/biotech_2nd_floor/second_floor_fume_hood_lab_spaces/lab_227b_control/occ'},
     'internal_temp_sensor': {'S': '#biotech/biotech_2nd_floor/second_floor_fume_hood_lab_spaces/lab_227b_control/zone/zone_temp/trend_log'},
     'hood_count': {'N': '1'},
     'floor': {'S': '2'},
     'lab': {'S': '227B'},
     'building': {'S': 'biotech'}}},
   'Biotech.Floor_3.Lab_327': {'M': {'room_occ_sensor': {'S': '#biotech/biotech_3rd_floor/third_floor_fume_hood_lab_spaces/lab_327_control/occ'},
     'internal_temp_sensor': {'S': '#biotech/biotech_3rd_floor/third_floor_fume_hood_lab_spaces/lab_327_control/zo

In [10]:
with table.batch_writer() as writer:
    writer.put_item(Item=get_hoods_dict())

response = dynamodb_client.get_item(
    TableName=TABLE_NAME, Key={"id": {"S": "hoods"}}
)
print("Hoods:")
display(response["Item"])

Hoods:


{'id': {'S': 'hoods'},
 'map': {'M': {'Biotech.Floor_2.Lab_257.Hood_1': {'M': {'flow_sensor': {'S': '#biotech/biotech_2nd_floor/second_floor_fume_hood_lab_spaces/lab_257_control/hoodvalve_flow'},
     'sash_position_sensor': {'S': '#biotech/biotech_2nd_floor/second_floor_fume_hood_lab_spaces/lab_257_control/hood_sash'},
     'floor': {'S': '2'},
     'lab': {'S': '257'},
     'hood': {'N': '1'},
     'building': {'S': 'biotech'}}},
   'Biotech.Floor_2.Lab_253B.Hood_1': {'M': {'flow_sensor': {'S': '#biotech/biotech_2nd_floor/second_floor_fume_hood_lab_spaces/lab_253b_control/hoodvalve_flow'},
     'sash_position_sensor': {'S': '#biotech/biotech_2nd_floor/second_floor_fume_hood_lab_spaces/lab_253b_control/hood_sash'},
     'floor': {'S': '2'},
     'lab': {'S': '253B'},
     'hood': {'N': '1'},
     'building': {'S': 'biotech'}}},
   'Biotech.Floor_4.Lab_417B.Hood_1': {'M': {'flow_sensor': {'S': '#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_417b_control/hoodvalve_flow

In [11]:
occupants = {
            "id": "occupants",
            "map": {
                "rf377@cornell.edu": {
                    "name": "Reid Fleishman",
                    "building": "biotech",
                    "lab_id": "Biotech.Floor_2.Lab_203b",
                },
                "ant46@cornell.edu": {
                    "name": "Aimee Tsang",
                    "building": "biotech",
                    "lab_id": "Biotech.Floor_2.Lab_203b",
                },
                "rw634@cornell.edu": {
                    "name": "Reinesse Wong",
                    "building": "biotech",
                    "lab_id": "Biotech.Floor_2.Lab_227b",
                },
            },
        }

with table.batch_writer() as writer:
    writer.put_item(Item=occupants)

response = dynamodb_client.get_item(
    TableName=TABLE_NAME, Key={"id": {"S": "occupants"}}
)
print("Occupants:")
display(response["Item"])

Occupants:


{'id': {'S': 'occupants'},
 'map': {'M': {'rf377@cornell.edu': {'M': {'name': {'S': 'Reid Fleishman'},
     'lab_id': {'S': 'Biotech.Floor_2.Lab_203b'},
     'building': {'S': 'biotech'}}},
   'rw634@cornell.edu': {'M': {'name': {'S': 'Reinesse Wong'},
     'lab_id': {'S': 'Biotech.Floor_2.Lab_227b'},
     'building': {'S': 'biotech'}}},
   'ant46@cornell.edu': {'M': {'name': {'S': 'Aimee Tsang'},
     'lab_id': {'S': 'Biotech.Floor_2.Lab_203b'},
     'building': {'S': 'biotech'}}}}}}

# Testing

In [52]:
dynamodb = boto3.resource('dynamodb', region_name="us-east-1")

table = dynamodb.Table('fumehoods')

response = table.scan(ProjectionExpression="id")
response['Items']

[{'id': 'buildings'}, {'id': 'occupants'}, {'id': 'hoods'}, {'id': 'labs'}]

In [51]:
table.delete_item(
    Key={
        'id': 'maggie'
    }
)

{'ResponseMetadata': {'RequestId': '1GHKH59R53SLGJ5RFIAD0MDMA7VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sat, 09 Nov 2024 05:30:16 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1GHKH59R53SLGJ5RFIAD0MDMA7VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [ ]:
response = dynamodb_client.get_item(
    TableName=TABLE_NAME, Key={"id": {"S": "labs"}}, ProjectionExpression="#map_alt.#lab", ExpressionAttributeNames={"#map_alt":"map", "#lab":"Biotech.Floor_4.Lab_417"}
)
display(response["Item"])

{'map': {'M': {'Biotech.Floor_4.Lab_417': {'M': {'room_occ_sensor': {'S': '#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_417_control/occ'},
     'internal_temp_sensor': {'S': '#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_417_control/zone/zone_temp/trend_log'},
     'hood_count': {'N': '1'},
     'floor': {'S': '4'},
     'lab': {'S': '417'},
     'building': {'S': 'biotech'}}}}}}

In [23]:
response = dynamodb_client.get_item(
    TableName=TABLE_NAME, Key={"id": {"S": "hoods"}}, ProjectionExpression="#map_alt.#hood", ExpressionAttributeNames={"#map_alt":"map", "#hood":"Biotech.Floor_4.Lab_417.Hood_1"}
)
display(response["Item"])

{'map': {'M': {'Biotech.Floor_4.Lab_417.Hood_1': {'M': {'flow_sensor': {'S': '#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_417_control/hoodvalve_flow'},
     'sash_position_sensor': {'S': '#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_417_control/hood_sash'},
     'floor': {'S': '4'},
     'lab': {'S': '417'},
     'hood': {'N': '1'},
     'building': {'S': 'biotech'}}}}}}